In [1]:
import sys
sys.path.append("../")
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from utils.inkml2img import convert_dir
from tqdm.auto import tqdm
from data.dataset import Im2LatexDataset
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from model.vit import ViT
import lightning as L
import wandb
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer, pre_tokenizers
from tokenizers.models import BPE
from model.transformer import EncoderTransformerBlock, DecoderAttention


In [2]:
img_dims = [256, 256]
model = ViT(img_dims, 16, n_embd=512)
data = Im2LatexDataset(path_to_data="../data/",
                       tokenizer="../data/tokenizer.json", img_dims=img_dims, batch_size=1, device=torch.device('mps'))
batch = next(iter(data.train))
model(batch[0])

tensor([[[ 0.6526, -0.3162,  0.2802,  ...,  0.6496, -1.0057, -0.1238],
         [ 0.7491,  0.1068,  1.2759,  ..., -0.5232,  0.1075, -0.9784],
         [ 0.5105, -0.5808,  0.4280,  ...,  0.2552,  0.5975, -1.0148],
         ...,
         [-0.2427, -0.5436,  0.1138,  ...,  0.8757, -1.2220, -0.3715],
         [ 0.4586,  0.4178, -0.2784,  ..., -0.2194,  0.1076, -0.9677],
         [ 0.3242,  0.5233, -0.8099,  ...,  1.0549, -0.2066,  0.5706]]],
       grad_fn=<ViewBackward0>)

In [3]:
HELP = DecoderAttention(512, 8)
inpu = torch.randn(1, 16, 512)
HELP(inpu, inpu, inpu).shape

torch.Size([1, 16, 512])

In [5]:
batch = next(iter(data.train))
tokenizer = PreTrainedTokenizerFast(tokenizer_file='../data/tokenizer.json')
tokens = list(map(lambda x: x.data, batch[1]['input_ids'][0]))
dec = [tokenizer.decode(tok) for tok in tokens]
#''.join([''.join(detok.split(' ')).replace('Ġ', ' ').strip() for detok in dec])
tokens

[tensor(1),
 tensor(99),
 tensor(477),
 tensor(100),
 tensor(19),
 tensor(99),
 tensor(138),
 tensor(104),
 tensor(102),
 tensor(101),
 tensor(272),
 tensor(99),
 tensor(139),
 tensor(103),
 tensor(105),
 tensor(272),
 tensor(96),
 tensor(2),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 

In [28]:
class TransformerDecoder(nn.Module):
    def __init__(self, n_embd, dropout=.75, block_size=256, num_blocks=6, num_heads=8):
        super().__init__()
        self.pos_embed = nn.Parameter(torch.zeros(1, block_size, n_embd))
        self.transformer_blocks = nn.ModuleList(
            [
                TransformerBlock(n_embd, num_heads, dropout) for _ in range(num_blocks)
            ]
        )
        

In [3]:
x = torch.randn(1, 256, 512)
x.mean(dim=1).shape

torch.Size([1, 512])